# Mini-Project: Geospatial Data Science

We obtain 3903 trajectories in total, which have been put in `trajectories_epsg_3857.txt`, where each row contains `img_id` and corresponding trajectory in EPSG:3857

Example:\
`Participant2_2019_March2019_Shot0146.jpg:-9665838.908 3954722.303,-9667308.043 3953792.864,-9667400.48 3953250.484,-9666948.894 3952362.957,-9666948.894 3952362.957`

### Data Downland and Loading

In [1]:
# get trajectories file from Github repo
import requests

url = 'https://github.com/gowtham30/geospatial_data_science_mini/raw/main/trajectories_epsg_3857.txt'
req = requests.get(url, allow_redirects=True)

open('trajectories_epsg_3857.txt', 'wb').write(req.content)

773390

In [2]:
!head -n 1 trajectories_epsg_3857.txt

Participant2_2019_March2019_Shot0146.jpg:-9665838.908 3954722.303,-9667308.043 3953792.864,-9667400.48 3953250.484,-9666948.894 3952362.957,-9666948.894 3952362.957


In [3]:
# read all trajectories from file into dictionary 'img2trajectory'
import os
img2trajectory = {} # {img_id:trajectory} 
n_line = 0

with open('trajectories_epsg_3857.txt', 'r') as f:
    for line in f:
      n_line += 1
      line_values = line.split(":")
      img_id = line_values[0]
      traj = line_values[1]
      img2trajectory[img_id] = []
      for x in traj.split(','):
        lat, lon = x.split(" ")
        img2trajectory[img_id].append((float(lat), float(lon)))

print("Number of trajectories: ", len(img2trajectory))

Number of trajectories:  3903


In [4]:
img2trajectory['Participant2_2019_March2019_Shot0146.jpg']

[(-9665838.908, 3954722.303),
 (-9667308.043, 3953792.864),
 (-9667400.48, 3953250.484),
 (-9666948.894, 3952362.957),
 (-9666948.894, 3952362.957)]

### Using PyProj package

Cartographic projections and coordinate transformations library, https://pyproj4.github.io/pyproj/stable/#

Use PyProj to transform one coordinate system to another.

In [5]:
! pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 4.2 MB/s 


In [8]:
# Example: Convert from EPSG:3857 to EPSG:4326, check here: https://epsg.io/ for more details.
from pyproj import Transformer

def convert_epsg(source_epsg, dist_epsg, x, y):
    transformer = Transformer.from_crs(source_epsg, dist_epsg)
    x, y = transformer.transform(x, y)
    return x, y

input_coords = (-9662852.767, 3962856.202)
convert_epsg('epsg:3857', 'epsg:4326', input_coords[0], input_coords[1])

(33.507460522863106, -86.8028832879271)

Remember in the bouns project we used https://www.georeferencer.com/ to get the coordinates. The format was epsg:3857

We need to convert to GPS coordinates in epsg:4326

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
img2trajectory_4326 = {}

photo_files = '/content/drive/MyDrive/Colab Notebooks/KYI' # path/to/uber_photos

from os import walk
filenames = next(walk(photo_files), (None, None, []))[2]  # [] if no file
for i in filenames:
  converted_coords_arr = []
  coords = img2trajectory[i]
  for input_coords in coords:
    converted_coords = convert_epsg('epsg:3857', 'epsg:4326', input_coords[0], input_coords[1])
    converted_coords_arr.append(converted_coords)
  img2trajectory_4326[i] = converted_coords_arr

### Visualizing a trajectory using Folium

Folium allows us to overlay plots on top of a map.

In [11]:
! pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir /content/maps
!mkdir /content/maps/maps_traj
!mkdir /content/maps/maps_marker

In [47]:
# plot a trajectory
import folium
import numpy as np
import time
from IPython.display import clear_output

for n in filenames:
  img_id = n
  traj = img2trajectory_4326[img_id]
  center = np.asarray(traj[0]) + np.asarray(traj[-1])
  center /= 2

  start_level = 13
  # if trajectory is not complete, reduce start_level until you see the entire trajectory
  # if trajectory is too small, increase start_level

  map = folium.Map(zoom_start=start_level, location=center)
  folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)


  map.save("/content/maps/maps_traj/" + img_id + ".html")
  # at each point on the trajectory, show a marker to check the order of points. 

  pos = 0
  while pos < len(traj):
      folium.Marker(traj[pos]).add_to(map)
      pos += 1
  map.save("/content/maps/maps_marker/" + img_id + ".html")

In [48]:
map

In [49]:
'''
# Below is an example of one image with an issue in the trajectory. We need to find such cases
#img_id = 'Participant2_2019_October2019_Shot0270.jpg'
# In case you want to check more images. Sample a random image and check the trajectory
# import random
# img_id = random.choice(list(img2trajectory.keys()))

good_bad_list = []

# convert coordinates
for img_id in filenames:
  traj = []
  for point in img2trajectory[img_id]:
    lat, lon = convert_epsg('epsg:3857', 'epsg:4326', point[0], point[1])
    traj.append((lat, lon))

  center = np.asarray(traj[0]) + np.asarray(traj[-1])
  center /= 2

  start_level = 13
  # if trajectory is not complete, reduce start_level until you see the entire trajectory
  # if trajectory is too small, increase start_level

  map = folium.Map(zoom_start=start_level, location=center)
  folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

  pos = 0
  while pos < len(traj):
      folium.Marker(traj[pos]).add_to(map)
      pos += 1
      display(map)
      clear_output(wait=True)
      time.sleep(0.5)
  while True:
    my_input = input("(good/bad)?")
    # check if d1a is equal to one of the strings, specified in the list
    if my_input in ['g', 'b']:
        # if it was equal - break from the while loop
        break

  good_bad_list.append(my_input)
'''

# manually clean data of bad mappings


'\n# Below is an example of one image with an issue in the trajectory. We need to find such cases\n#img_id = \'Participant2_2019_October2019_Shot0270.jpg\'\n# In case you want to check more images. Sample a random image and check the trajectory\n# import random\n# img_id = random.choice(list(img2trajectory.keys()))\n\ngood_bad_list = []\n\n# convert coordinates\nfor img_id in filenames:\n  traj = []\n  for point in img2trajectory[img_id]:\n    lat, lon = convert_epsg(\'epsg:3857\', \'epsg:4326\', point[0], point[1])\n    traj.append((lat, lon))\n\n  center = np.asarray(traj[0]) + np.asarray(traj[-1])\n  center /= 2\n\n  start_level = 13\n  # if trajectory is not complete, reduce start_level until you see the entire trajectory\n  # if trajectory is too small, increase start_level\n\n  map = folium.Map(zoom_start=start_level, location=center)\n  folium.PolyLine(traj, color=\'red\', weight=5, opacity=0.8).add_to(map)\n\n  pos = 0\n  while pos < len(traj):\n      folium.Marker(traj[pos]).a

In [50]:
'''
for i in range(len(good_bad_list)):
  if good_bad_list[i] == "b":
    print(filenames[i])
    print(i)
  '''


'\nfor i in range(len(good_bad_list)):\n  if good_bad_list[i] == "b":\n    print(filenames[i])\n    print(i)\n  '

### Fast map matching (FMM)*

Reference: Can Yang & Gyozo Gidofalvi (2018) Fast map matching, an algorithm
integrating hidden Markov model with precomputation, International Journal of Geographical Information Science, 32:3, 547-570, DOI: 10.1080/13658816.2017.1400548

https://fmm-wiki.github.io/

As you may have noticed, the trajectories are not aligned with the underlying road network. i.e., a line between two consecutive points is a straight line and does not follow the actual roads.

To do match those points to the road network, and add back intermediate points, we use FMM. First, we need to install it.

**Install Prerequisites**

In [51]:
# install FMM "requirements" following https://fmm-wiki.github.io/docs/installation
# You need to choose the right OS on that page
# For example, on Ubuntu you can run
# !sudo apt-get install libboost-dev libboost-serialization-dev gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig python-dev

In [52]:
# Install all the requirements with:
! sudo apt-get install libboost-dev libboost-serialization-dev \
gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig python-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-dev is already the newest version (1.65.1.0ubuntu1).
make is already the newest version (4.1-9.1ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
gdal-bin is already the newest version (2.2.3+dfsg-2).
libboost-serialization-dev is already the newest version (1.65.1.0ubuntu1).
libgdal-dev is already the newest version (2.2.3+dfsg-2).
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libexpat1-dev is already the newest version (2.2.5-3ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


**Install FMM**

In [53]:
!git clone https://github.com/cyang-kth/fmm.git

Cloning into 'fmm'...
remote: Enumerating objects: 5162, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 5162 (delta 14), reused 35 (delta 10), pack-reused 5119
Receiving objects: 100% (5162/5162), 15.33 MiB | 12.38 MiB/s, done.
Resolving deltas: 100% (3062/3062), done.


In [22]:
# it may not work and requires for password, etc.
# in this case, go to your folder using console to compile and install FMM

import os
# change working directory
os.chdir("fmm")

if not os.path.exists('build'):
  os.mkdir('build')
# ! mkdir build
os.chdir("build")
# ! cd build
! cmake ..
! make -j4
! sudo make install

-- CMAKE version 3.22.5
-- Set CMP0074 state to NEW
-- Set CMP0086 state to NEW
-- Set CMP0078 state to NEW
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- 
No conda environment found in PATH!
PATH=/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin

-- Could NOT find Conda (missing: CONDA_PREFIX) 
-- Non conda exist, search library in default path
-- Found GDAL: /usr/lib/libgdal.so (found suitable version "2.2.3", minimum required is

In [54]:
# Verfication of installation
!fmm

[info][fmm_app_config.cpp:49 ] Start reading FMM configuration from arguments
fmm argument lists:
--ubodt (required) <string>: Ubodt file name
--network (required) <string>: Network file name
--network_id (optional) <string>: Network id name (id)
--source (optional) <string>: Network source name (source)
--target (optional) <string>: Network target name (target)
--gps (required) <string>: GPS file name
--gps_id (optional) <string>: GPS id name (id)
--gps_x (optional) <string>: GPS x name (x)
--gps_y (optional) <string>: GPS y name (y)
--gps_timestamp (optional) <string>: GPS timestamp name (timestamp)
--gps_geom (optional) <string>: GPS geometry name (geom)
--gps_point (optional): if specified read input data as gps point, otherwise (default) read input data as trajectory
--output (required) <string>: Output file name
--output_fields (optional) <string>: Output fields
  opath,cpath,tpath,mgeom,pgeom,
  offset,error,spdist,tp,ep,length,duration,speed,all
-k/--candidates (optional) <int>

In [55]:
# Change to the parent folder which contains fmm_test.py
if os.getcwd() != "/content/fmm/example/python":
  os.chdir("/content/fmm/example/python")
os.system('python fmm_test.py')

256

In [25]:
# Install osmnx package to prepare the network, that will be used as input to FMM
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 3.0 MB/s 
     |████████████████████████████████| 11.2 MB 26.9 MB/s 
     |████████████████████████████████| 1.0 MB 55.8 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 1.0 MB 23.9 MB/s 
     |████████████████████████████████| 16.7 MB 32.6 MB/s 
     |████████████████████████████████| 930 kB 44.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

In [26]:
import osmnx as ox
import time
from shapely.geometry import Polygon
import os
import numpy as np
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = np.arange(0, gdf_edges.shape[0], dtype='int')
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)

# !!! Download by a bounding box # !!!
# -------------------------------------------------
bounds = (-86.9671,-86.5901,33.3472,33.6598)
# -------------------------------------------------
x1,x2,y1,y2 = bounds
boundary_polygon = Polygon([(x1,y1),(x2,y1),(x2,y2),(x1,y2)])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
start_time = time.time()
save_graph_shapefile_directional(G, filepath='./network-new')
print("--- %s seconds ---" % (time.time() - start_time))

osmnx version 1.1.2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


--- 35.31191444396973 seconds ---


## Steps to run FMM after installation: 
1- Prepare the network using osmnx \
2- Run the match(...) function

In [27]:
# copy network folder to /content/fmm/example/python/network-new
!mv '/content/network-new' '/content/fmm/example/python/network-new'

mv: cannot stat '/content/network-new': No such file or directory


In [28]:
# match(...) function

def match(path,points):
  network = Network(path,"fid","u","v")
  graph = NetworkGraph(network)
  print (graph.get_num_vertices())
  model = STMATCH(network,graph)
  wkt  = str(points)
  config = STMATCHConfig()
  config.k = 4
  config.gps_error = 1
  config.radius = 0.4
  config.vmax = 30;
  config.factor = 1.5
  result = model.match_wkt(wkt,config)
  print (type(result))
  # print ("Opath ",list(result.opath))
  # print ("Cpath ",list(result.cpath))
  # print ("WKT ",result.mgeom.export_wkt())
  return result.mgeom.export_wkt()

In [29]:
def match_trajs(network_path, traj):
  """
  match [traj] on [network_path]. [traj] should be [(lon1, lat1), (lon2, lat2), ...]
  Output: matched traj as "LINESTRING(-86.797211 33.499194,-86.7973..."
  """
  from shapely.geometry import LineString
  # import geopandas !!

  traj_ = LineString(traj)
  return match(network_path, traj_)

In [30]:
!mkdir /content/maps/maps_fmm
!mkdir /content/maps/maps_heat

In [56]:
from folium.plugins import HeatMap

for n in list(img2trajectory_4326.keys()):
  print(n)
  img_id = n
  traj = img2trajectory_4326[img_id]
  print("Original Trajectory: ", traj)
  # swap (lat, lon) --> (lon, lat) as FMM requires lon before lat
  traj_m = [(sub[1], sub[0]) for sub in traj]
  fmm_traj = match_trajs('/content/fmm/example/python/network-new/edges.shp', traj_m)
  # remove LINESTRING and extra parentheses
  fmm_traj = fmm_traj.split("LINESTRING(")[1]
  fmm_traj = fmm_traj.split(")")[0]

  # plot both trajctories, the original and the matched one.

  matched_traj = []
  for c in fmm_traj.split(","):
      lon, lat = c.split(" ")
      matched_traj.append((float(lat), float(lon)))

  center = np.asarray(matched_traj[0]) + np.asarray(matched_traj[-1])
  center /= 2

  start_level = 13
  # if trajectory is not complete, reduce start_level until you see the entire trajectory
  # if trajectory is too small, increase start_level

  map = folium.Map(zoom_start=start_level, location=center)
  folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)
  folium.PolyLine(matched_traj, color='green', weight=5, opacity=0.8).add_to(map)

  map.save("/content/maps/maps_fmm/" + img_id + ".html")


  map = folium.Map(zoom_start=10, height=700, location=[33.5186, -86.8104])
  HeatMap(matched_traj, blur=5, radius=5).add_to(map)
  map.save("/content/maps/maps_heat/" + img_id + ".html")

Participant5_2019_March2019_Shot0031.jpg
Original Trajectory:  [(33.57670612292052, -86.75582311039061), (33.54452085295345, -86.76047591643841), (33.528065966263675, -86.80526158662494), (33.48445925741484, -86.78648250323978), (33.44164034709229, -86.73120753785724), (33.395153007097775, -86.7842425810283), (33.41177643696678, -86.80655855626584), (33.41177643696678, -86.80655855626584)]
32326
<class 'fmm.PyMatchResult'>
uber_imgs_Participant6_November2021_Shot0021.jpg
Original Trajectory:  [(33.562020493459315, -86.75122555971596), (33.54406997300196, -86.75443113492526), (33.51994456760529, -86.82530699811666), (33.38148700657165, -86.80791297316917), (33.38148700657165, -86.80791297316917)]
32326
<class 'fmm.PyMatchResult'>
uber_imgs_Participant6_March2021_Shot0147.jpg
Original Trajectory:  [(33.48780034756422, -86.86155593324244), (33.48350487203042, -86.82632447492323), (33.393988649241926, -86.78381797434295), (33.441331857390225, -86.72990594189264), (33.43298237700339, -86.71

In [57]:
map

We can also visualize the trajectories using Folium HeatMap

In [58]:
# to understand the density of trips, we can decompose trajectories into points and plot
# decompose your trajectories and add to a new list. Then use Heatmap to plot

import random

matched_traj = []
for n in img2trajectory_4326:
  img_id = n
  traj = img2trajectory_4326[img_id]
  print("Original Trajectory: ", traj)
  # swap (lat, lon) --> (lon, lat) as FMM requires lon before lat
  traj_m = [(sub[1], sub[0]) for sub in traj]
  fmm_traj = match_trajs('/content/fmm/example/python/network-new/edges.shp', traj_m)
  # remove LINESTRING and extra parentheses
  fmm_traj = fmm_traj.split("LINESTRING(")[1]
  fmm_traj = fmm_traj.split(")")[0]
  print("Matched Trajectory: ", fmm_traj)
  if len(fmm_traj) == 0:
    continue
  for c in fmm_traj.split(","):
      lon, lat = c.split(" ")
      matched_traj.append((float(lat), float(lon)))


Original Trajectory:  [(33.57670612292052, -86.75582311039061), (33.54452085295345, -86.76047591643841), (33.528065966263675, -86.80526158662494), (33.48445925741484, -86.78648250323978), (33.44164034709229, -86.73120753785724), (33.395153007097775, -86.7842425810283), (33.41177643696678, -86.80655855626584), (33.41177643696678, -86.80655855626584)]
32326
<class 'fmm.PyMatchResult'>
Matched Trajectory:  -86.755711 33.576549,-86.755722 33.576541,-86.755988 33.576312,-86.756237 33.576105,-86.756357 33.575969,-86.756521 33.575769,-86.756709 33.575468,-86.756847 33.575189,-86.756958 33.574882,-86.757044 33.574496,-86.75707 33.574152,-86.757061 33.573766,-86.757061 33.573445,-86.757061 33.573123,-86.757061 33.572837,-86.757036 33.572565,-86.757001 33.572315,-86.756898 33.571914,-86.756804 33.571585,-86.756641 33.571063,-86.756529 33.570691,-86.756402 33.570329,-86.756326 33.57003,-86.75628 33.569847,-86.75628 33.569447,-86.75634 33.569139,-86.756478 33.568825,-86.75662 33.568594,-86.757393 

In [59]:
Amap = folium.Map(zoom_start=10, height=700, location=[33.5186, -86.8104])
HeatMap(matched_traj, blur=5.5, radius=6).add_to(map)
map.save("/content/cumulative_heatmap.html")
map

In [35]:
# The End

In [42]:
!pwd

/content/fmm/example/python


In [61]:
%%shell
jupyter nbconvert --to html "Uber_Vis.ipynb"

[NbConvertApp] Converting notebook Uber_Vis.ipynb to html
/usr/local/lib/python3.7/dist-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.colab-display-data+json']) is not able to be represented.
  mimetypes=output.keys())
[NbConvertApp] Writing 1052253 bytes to Uber_Vis.html


In [46]:
!tar chvfz notebook.tar.gz /content/maps


tar: Removing leading `/' from member names
/content/maps/
/content/maps/maps_traj/
/content/maps/maps_traj/Participant3_2021_March2021_Shot0005.jpg.html
/content/maps/maps_traj/Participant2_2019_May2019_Shot0142.jpg.html
/content/maps/maps_traj/Participant2_2021_November2021_Shot0152.jpg.html
/content/maps/maps_traj/Participant2_2021_September2021_Shot0376.jpg.html
/content/maps/maps_traj/uber_imgs_Participant6_March2021_Shot0147.jpg.html
/content/maps/maps_traj/uber_imgs_Participant6_March2021_Shot0006.jpg.html
/content/maps/maps_traj/uber_imgs_Participant6_November2021_Shot0021.jpg.html
/content/maps/maps_traj/Participant2_2019_September2019_Shot0142.jpg.html
/content/maps/maps_traj/uber_imgs_Participant6_October2021_Shot0060.jpg.html
/content/maps/maps_traj/Participant2_2019_November2019_Shot0125.jpg.html
/content/maps/maps_traj/Participant2_2019_October2019_Shot0257.jpg.html
/content/maps/maps_traj/uber_imgs_Participant6_November2021_Shot0086.jpg.html
/content/maps/maps_traj/Parti